In [2]:
from feature_extraction.orb import ORB
import os

import cv2
from matplotlib import pyplot as plt

from sklearn.cluster import KMeans

DATA_PATH="../data/Caltech20"
TRAINING_DIR=os.path.join(DATA_PATH, "training")
TEST_DIR=os.path.join(DATA_PATH, "testing")

classes = []

features = {}
all_descriptor = []

for class_name in sorted(os.listdir(TRAINING_DIR)):
    classes.append(class_name)
    features[class_name] = {}
    class_dir = os.path.join(TRAINING_DIR, class_name)
    for file_name in sorted(os.listdir(class_dir)):
        filename = os.path.join(class_dir, file_name)
        kp, des = ORB.compute_file(filename)
        if des is not None:
            des = des[:32]
            features[class_name][file_name] = des
            all_descriptor.extend(des)

In [3]:
kmeans = KMeans(10)
kmeans.fit(all_descriptor)
centers = kmeans.cluster_centers_

In [5]:
from feature_quantization.bag_of_visual_words import get_histogram

histograms = {}
for c_name in features:
    histograms[c_name] = []
    for img in features[c_name]:
        histograms[c_name].append(get_histogram(features[c_name][img], centers))

In [6]:
histograms['airplanes'][0]

array([2., 1., 5., 4., 1., 0., 5., 4., 5., 5.])

In [7]:
X = []
Y = []
for c in histograms:
    X.extend(histograms[c])
    Y.extend([c] * len(histograms[c]))
    

In [8]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X, Y)


DecisionTreeClassifier()

In [10]:
test_classes = []
test_features = {}

testX = []
testY = []

for class_name in sorted(os.listdir(TEST_DIR)):
    class_dir = os.path.join(TEST_DIR, class_name)
    for file_name in sorted(os.listdir(class_dir)):
        filename = os.path.join(class_dir, file_name)
        kp, des = ORB.compute_file(filename)
        if des is not None:
            des = des[:32]
            histogram = get_histogram(des, centers)
            testX.append(histogram)
            testY.append(class_name)

classifier.score(testX, testY)

0.07105263157894737